<a href="https://colab.research.google.com/github/tkornuta-semiotic/2023-05-AI-workshop/blob/main/Image_Recognition_with_HuggingFace_and_ResNet_FineTuning_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>